In [32]:
import MeCab
import CaboCha
import pandas as pd
from tqdm import tqdm 
import regex
import re

In [25]:
def get_word(tree, chunk):
    surface = ''
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞':
            surface += token.surface
        elif features[0] == '形容詞':
            surface += features[6]
            break
        elif features[0] == '動詞':
            surface += features[6]
            break
    return surface

def get_2_words(line):
    cp = CaboCha.Parser('-f1')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_word(tree, to_chunk)
            tuples.append((from_surface, to_surface))
    return tuples

In [46]:
#口コミを１文ずつに切り分けし、その文章内を係り受け解析し（単語A,単語B）にkawaii_wordがあればリストに加える
def taples_kawaii(txt):
    kawaii_word = ['可愛い','かわいい','カワイイ','美人','綺麗','きれい','美しい','美女']
    sen = regex.split(r'(?<=[。？])(?!$)', txt, flags=regex.VERSION1)
    #sen = re.split(r'(?<=[。？])(?!$)', txt)
    add_list=[]
    for i in sen:
        tuples = get_2_words(i)
        for j in tuples:
            for k in kawaii_word:
                if k in j:
                    add_list.append(j)
    return add_list

In [27]:
#形態素解析するための関数を定義
tagger = MeCab.Tagger('-Owakati')#タグはMeCab.Tagger（neologd辞書）を使用したい
tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [42]:
#kawaii_wordが含まれた（単語A,単語B）をさらに形態素解析し、tennin_wordが含まれていたらフラグ立てをする
def tennin_hantei(i):

    txt = tokenize(i, 2, 10000, True)

    vector_list =[]

    a = 0

    tennin_word = ['店員','女性店員','スタッフ','女将','女性','店主','助手','奥さん','奥様','店長']
    for k in txt:
        if k in tennin_word:
            a+=1
        else:
            a+=0        
    if a >=1:
        kawaii_flg = 1
    else:
        kawaii_flg = 0

    return kawaii_flg

In [50]:
df = pd.read_csv('all_results.csv')
df['store_score'].astype(float)
df = df[df['store_score'] > 4.7]
df = df.dropna(subset=['dinner_review'])
df = df.drop(['detail', 'lunch_review', 'score'], axis=1)
df.head()

,dinner_review,store_name,store_score
7716,\n 六本木の【鮨 さいとう】さんに伺いました。地下鉄六本木一丁目駅の3番出口を出て目の前のエスカレターで1階に上がると左手にお店はあります。今回も斎藤さんのお席でつまみからにぎりのおまかせを頂きました。つまみ○白海老 富山産。 生姜醤油で頂きました。 ねっとり甘くてとても美味しい。○蒸し鮑 大原産。 うま味濃厚。余韻も素晴らしい。○蛸 佐島産。 弾力のある食感。 噛めばうま味が溢れます。○カツオ 気仙沼産。 皮目カリッと脂が乗ってうま味が濃く美味しい。☆ウニの三点盛り○バフンウニ 余市産塩水ウニ。 とろけてうま味が口に広がります。○ムラサキウニ 余市産。 甘味が濃かった。○赤ウニ 唐津産。 濃厚なうま味があり最高でした。○のどぐろ焼き 高知産。 とろけてジュワーと脂が溢れます。 安定の美味しさ。 にぎり●ホシガレイ 岩手産。 昆布〆。 香り良くうま味が濃く余韻が...,鮨 さいとう,4.72
7717,\n この日は久々の鮨さいとうへ(*･ω･)ﾉ今回はちょっと特別な会。普段こちらはワインを合わせる様な事は無いらしいのですが、今回は鮨にワインをマリアージュさせる趣旨でした。橋場さんの席でしたが、最近はツマミなどは橋場さんの裁量の範囲が広がっているそう。以前来た時より格段に美味しくなっていましたね〜〜。鰹の皮目がパリッとしてコントラストが素晴らしい。煮ハマグリも上品な味わい。斎藤さんの席よりこちらを推す人もいる理由が分かりました(*･ω･)ﾉ今回のメニューつまみ⭐️白海老⭐️タコ⭐️アワビ⭐️鰹⭐️煮ハマグリ⭐️のどぐろ白湯⭐️漬物、ガリ握り⭐️ヒラメ⭐️いさき⭐️コハダ⭐️赤身⭐️中トロ⭐️大トロ⭐️イカ⭐️車海老⭐️アジ⭐️雲丹⭐️穴子⭐️巻物⭐️玉子\n,鮨 さいとう,4.72
7718,\n 友人が予約を取ってくれたところに飛び入りで参加してきました。こちらのお店は移転前前から通っていますが、最近は半年に1度程度の予約のみになってしまったため、すっかり足が遠のいてしまっています。お弟子さんが握るのならともかく、斎藤氏ご本人に握ってもらうのは本当に困難になってしまったからです。斎藤氏のお鮨はもう世界レベルで認知されてしまったので今さらどうこうは申し上げません。私が『かねさか』系のお鮨が好きとうのもあるかもしれませんが、素晴らしい、さすがの一言。こう申し上げると「え？孤高のグルメさん、鮨わかってないの？」などと思われるかもしれませんが、私はかねさか系の握り鮨ではこはだや鯵（あじ）とまぐろ赤身の漬けが好きなんですよね（これはかねさか系のお鮨をよくご存じの方はお分かりだと思います）。スタッフの対応もこれだけ繁盛しているにも関わらず驕り高ぶらず及第点レベル...,鮨 さいとう,4.72
7723,\n 六本木の【鮨さいとう】さんに伺いました。今回も齋藤さんのお席でつまみからにぎりのおまかせを頂きました。つまみ○白海老 生姜醤油で頂きました。 白海老の甘味と生姜醤油の風味がベストマッチ。○鯛の子 木の芽が添えられる。 口の中に広がる鯛の子の上品な甘味と旨味が堪らない味わい。○蒸し鮑 弾けるような感じの柔らかい食感。 香りと旨味最高。○蛸 とろけるような柔らかさ。旨味が素晴らしくとても美味しい。○ホタルイカ 香り素晴らしく、濃厚な旨味。絶品でした。○カツオ 提供直前に齋藤さんが炙っていました。 皮目カリカリ、身はしっとりして香りと旨味が素晴らしい。○あん肝 青紫蘇がかけられる。 とろける食感。旨味濃厚。 青紫蘇で甘味が引き立ちとても美味しい。○のどぐろと水ナス 対馬産、紅瞳。 脂の甘味と旨味が口の中に溢れます。 添えられた大根おろしは辛味が出ないようにゆっくり...,鮨 さいとう,4.72
7724,\n 六本木一丁目の【鮨 さいとう】さんに伺いました。いつも通りつまみからにぎりのおまかせで頂きました。つまみ○イクラ 上品な味。旨味があり美味しい。○鮑 千葉県大原産。 肝ソースで頂きます。 旨味濃い。 肝ソースが舌触り滑らかで海藻の香りが強く絶品。○蛸 佐島産。 柔らかく旨味強い。○鰹 勝浦産。 香り、身質良く濃厚な味わい。○鱈の白子 提供温度抜群。 ねっとりした舌触りクリーミーでとても美味しい。 ○毛蟹 噴火湾産。 毛蟹の甘味と旨味が濃く美味しい。○のどぐろ 脂の美味しさを堪能。 山芋のわさび漬けが添えられましたが、サクサクで香りが良くとても美味しかった。にぎり●鰈 青森産。 食感柔らかく、旨味濃い。●ブリ 余市産。 上質な脂で濃厚な旨味。とても美味しい。●コハダ 天草産。 香り素晴らしく。旨味が口の中に広がる。●赤身 大間産。 肉の様な食感で噛むと旨味が溢...,鮨 さいとう,4.72


In [47]:
pd.set_option("display.max_colwidth", 400)
list_sentence = df['dinner_review'].tolist()

new_list=[]
for txt in tqdm(list_sentence):
    new_list.append(taples_kawaii(txt))
kawaii_flg_list2=[]
kawaii_flg_list3=[]
for k in new_list:
    kawaii_flg_list=[]
    if len(k)==0:
        kawaii_flg_list.append(0)
    else: 
        for i in k:
                kawaii_flg_list.append(tennin_hantei(i))
                if tennin_hantei(i)==1:
                    kawaii_flg_list3.append(sum(kawaii_flg_list))

df['kawaii_flg']=kawaii_flg_list3
df_ramen = df.groupby(['store_name','store_score'])['dinner_review'].apply(list).apply(' '.join).reset_index().sort_values('store_score', ascending=False)
df_ramen2 = df.groupby(['store_name','store_score'])['kawaii_flg'].sum().reset_index().sort_values('kawaii_flg', ascending=False)
df_ramen2



  0%|                                                                                         | 0/1078 [00:00<?, ?it/s]

  0%|                                                                                 | 1/1078 [00:00<08:40,  2.07it/s]

  0%|▏                                                                                | 3/1078 [00:00<07:08,  2.51it/s]

  0%|▎                                                                                | 4/1078 [00:01<07:08,  2.50it/s]

  0%|▍                                                                                | 5/1078 [00:01<06:47,  2.63it/s]

  1%|▍                                                                                | 6/1078 [00:01<05:37,  3.17it/s]

  1%|▌                                                                                | 7/1078 [00:02<05:14,  3.41it/s]

  1%|▌                                                                                | 8/1078 [00:02<05:19,  3.34it/s]

  1%|▋                        

  8%|██████▌                                                                         | 89/1078 [00:18<02:08,  7.68it/s]

  8%|██████▊                                                                         | 91/1078 [00:18<01:48,  9.08it/s]

  9%|██████▉                                                                         | 93/1078 [00:19<03:52,  4.23it/s]

  9%|██████▉                                                                         | 94/1078 [00:19<03:15,  5.05it/s]

  9%|███████                                                                         | 95/1078 [00:19<03:45,  4.35it/s]

  9%|███████                                                                         | 96/1078 [00:20<03:17,  4.97it/s]

  9%|███████▏                                                                        | 97/1078 [00:20<04:38,  3.52it/s]

  9%|███████▎                                                                        | 98/1078 [00:21<06:33,  2.49it/s]

  9%|███████▎                   

 15%|███████████▊                                                                   | 161/1078 [00:37<04:47,  3.19it/s]

 15%|███████████▊                                                                   | 162/1078 [00:37<04:21,  3.51it/s]

 15%|███████████▉                                                                   | 163/1078 [00:37<03:46,  4.03it/s]

 15%|████████████▏                                                                  | 166/1078 [00:37<02:49,  5.39it/s]

 16%|████████████▎                                                                  | 168/1078 [00:38<02:23,  6.35it/s]

 16%|████████████▌                                                                  | 172/1078 [00:38<01:50,  8.20it/s]

 16%|████████████▊                                                                  | 174/1078 [00:38<01:42,  8.83it/s]

 16%|████████████▉                                                                  | 176/1078 [00:38<01:27, 10.26it/s]

 17%|█████████████              

 24%|███████████████████▏                                                           | 261/1078 [00:55<00:52, 15.65it/s]

 24%|███████████████████▎                                                           | 263/1078 [00:55<00:51, 15.78it/s]

 25%|███████████████████▍                                                           | 265/1078 [00:55<00:57, 14.24it/s]

 25%|███████████████████▌                                                           | 267/1078 [00:55<00:57, 14.20it/s]

 25%|███████████████████▋                                                           | 269/1078 [00:55<00:58, 13.74it/s]

 25%|███████████████████▊                                                           | 271/1078 [00:56<01:05, 12.34it/s]

 25%|████████████████████                                                           | 273/1078 [00:56<01:02, 12.85it/s]

 26%|████████████████████▏                                                          | 275/1078 [00:56<01:01, 13.16it/s]

 26%|████████████████████▎      

 38%|██████████████████████████████▏                                                | 412/1078 [01:09<01:31,  7.31it/s]

 38%|██████████████████████████████▎                                                | 414/1078 [01:09<01:19,  8.35it/s]

 39%|██████████████████████████████▍                                                | 416/1078 [01:10<01:10,  9.35it/s]

 39%|██████████████████████████████▋                                                | 418/1078 [01:10<01:00, 10.93it/s]

 39%|██████████████████████████████▊                                                | 420/1078 [01:10<01:02, 10.54it/s]

 39%|██████████████████████████████▉                                                | 422/1078 [01:10<00:56, 11.66it/s]

 39%|███████████████████████████████                                                | 424/1078 [01:10<00:55, 11.76it/s]

 40%|███████████████████████████████▏                                               | 426/1078 [01:10<00:53, 12.30it/s]

 40%|███████████████████████████

...

 67%|████████████████████████████████████████████████████▊                          | 721/1078 [02:01<01:25,  4.18it/s]

 67%|████████████████████████████████████████████████████▉                          | 723/1078 [02:01<01:21,  4.34it/s]

 67%|█████████████████████████████████████████████████████                          | 724/1078 [02:02<01:44,  3.38it/s]

 67%|█████████████████████████████████████████████████████▏                         | 725/1078 [02:02<01:36,  3.66it/s]

 67%|█████████████████████████████████████████████████████▏                         | 726/1078 [02:02<01:24,  4.16it/s]

 67%|█████████████████████████████████████████████████████▎                         | 727/1078 [02:02<01:15,  4.65it/s]

 68%|█████████████████████████████████████████████████████▎                         | 728/1078 [02:03<02:12,  2.64it/s]

 68%|█████████████████████████████████████████████████████▍                         | 729/1078 [02:04<02:26,  2.38it/s]

 68%|███████████████████████████

 74%|██████████████████████████████████████████████████████████▏                    | 794/1078 [02:21<01:05,  4.36it/s]

 74%|██████████████████████████████████████████████████████████▎                    | 795/1078 [02:21<01:10,  4.03it/s]

 74%|██████████████████████████████████████████████████████████▎                    | 796/1078 [02:21<01:06,  4.26it/s]

 74%|██████████████████████████████████████████████████████████▍                    | 797/1078 [02:21<00:56,  4.94it/s]

 74%|██████████████████████████████████████████████████████████▌                    | 799/1078 [02:22<00:46,  6.05it/s]

 74%|██████████████████████████████████████████████████████████▊                    | 802/1078 [02:22<00:35,  7.79it/s]

 75%|██████████████████████████████████████████████████████████▉                    | 804/1078 [02:22<00:28,  9.47it/s]

 75%|███████████████████████████████████████████████████████████▏                   | 808/1078 [02:22<00:23, 11.58it/s]

 75%|███████████████████████████

 85%|███████████████████████████████████████████████████████████████████▍           | 920/1078 [02:36<00:26,  5.88it/s]

 85%|███████████████████████████████████████████████████████████████████▍           | 921/1078 [02:36<00:26,  5.83it/s]

 86%|███████████████████████████████████████████████████████████████████▌           | 922/1078 [02:37<00:53,  2.92it/s]

 86%|███████████████████████████████████████████████████████████████████▋           | 923/1078 [02:38<01:06,  2.33it/s]

 86%|███████████████████████████████████████████████████████████████████▋           | 924/1078 [02:38<00:54,  2.81it/s]

 86%|███████████████████████████████████████████████████████████████████▊           | 926/1078 [02:38<00:42,  3.54it/s]

 86%|███████████████████████████████████████████████████████████████████▉           | 927/1078 [02:38<00:39,  3.82it/s]

 86%|████████████████████████████████████████████████████████████████████           | 928/1078 [02:39<00:59,  2.51it/s]

 86%|███████████████████████████

 93%|████████████████████████████████████████████████████████████████████████▌     | 1003/1078 [02:57<00:14,  5.01it/s]

 93%|████████████████████████████████████████████████████████████████████████▋     | 1004/1078 [02:57<00:15,  4.87it/s]

 93%|████████████████████████████████████████████████████████████████████████▋     | 1005/1078 [02:57<00:13,  5.32it/s]

 93%|████████████████████████████████████████████████████████████████████████▊     | 1006/1078 [02:58<00:13,  5.16it/s]

 94%|████████████████████████████████████████████████████████████████████████▉     | 1008/1078 [02:58<00:11,  6.19it/s]

 94%|█████████████████████████████████████████████████████████████████████████     | 1010/1078 [02:58<00:09,  6.82it/s]

 94%|█████████████████████████████████████████████████████████████████████████▏    | 1012/1078 [02:58<00:09,  6.80it/s]

 94%|█████████████████████████████████████████████████████████████████████████▎    | 1013/1078 [02:58<00:09,  6.82it/s]

 94%|███████████████████████████

ValueError: Length of values does not match length of index